# **Single Fingerprint Deep Classification**
### **JCE - Software Engineering Final Project** 
##### ***By Kobi Amsellem & Zohar Kedem***

##### In this study we want to discovare if Deep Convolutional Neural Network can classify single fingerprint image to find the owner: 
##### **1. Gender** - 2 classes (male/female).
##### **2. Finger name** - 10 classes (right-thumb, ..., right-thumb, ...)
##### **3. Fingerprint type** - 5 classes (left loop, whirl, right loop, tented arch, arch)
##### **4. Same Person** - 2 classes (Same, Different) *(whether or not two fingerprints belong to the same personperson)* 


In [ ]:
import os
import copy
import datasets as ds
import FPMLmodule.utils as utils
import tensorflow as tf
import FPMLmodule.backbones as backbones
import FPMLmodule.classifiers as classifiers
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from FPMLmodule.fpml import FPML 
from pathlib import Path

print('TensorFlow Version:', tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
dbStudyOutPath ='./out/{}/'

weightsRN50 = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsMNV2 = "./weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
weightsENB2 = "./weights/efficientnetb2_notop.h5"
weightsINCEPTIONV3 = "./weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsXCEPTION = "./weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

#### **Model Search** 
##### Main method to search after the best model

in this section we define the ````researchBestModel````, a method to search and compare between several different models and hyper parameters and choose the best one for specific dataset.

##### **Stages:**
##### **1.** Backbone Transfer Learning mode comparison - (a) Pretraind wheights and Untrainable, (b) Pretraind wheights and Trainable, (c) Initialize wheight and Trainable.
##### **2.** Loss function comparison. 
##### **3.** Learning rates comparison. 
##### **4.** Optimizers comparison. 
##### **5.** Backbone comparison. 
##### **6.** Train best configuration for extra epochs on train+validation datasets

In [ ]:
def researchBestModel(configureDS, datasets, epochsSearch, epochsBest, path, optimizers, learningRates, defaultHypers, losses, defaultBackbone, backbonesForSearch, verbose=1):
    
    trainDs, testDs, valDs = datasets
    imgDim = configureDS.inputDim
    nbClasses = len(configureDS.classNames)
    outFilePath = path + 'ablationHistory.csv'
    
    print('Ablation for', configureDS.name)
    # --------------------------------------------------
    # ---- Stage 1 - Compare Transfer Learning Mode ----
    # --------------------------------------------------
    trainingModeStudy =  {
        'DNN-TransferLearningMode' : {
            "architecture": {
                "backbone": [
                    defaultBackbone(imgDim, weights=weightsMNV2, trainable=False, name='PT&Untrainable'),
                    defaultBackbone(imgDim, weights=weightsMNV2, trainable=True, name='PT&Trainable'),
                    defaultBackbone(imgDim, weights=None, trainable=True, name='IN&Trainable')
                    ],
                "classfier": classifiers.DefaultClassifier(nbClasses, "softmax"),
                "inputLayer": None, 
                "inputDim": imgDim
            },
            "hyperparameters":defaultHypers
        }
    }
    
    trainingModeHistories = utils.researchStudies(trainDs, valDs, trainingModeStudy, epochsSearch, verbose)
    bestTrainMode = utils.getBestStudyFromHistories(trainingModeHistories)
    trainable = 'Trainable' in bestTrainMode
    pretrained = 'PT' in bestTrainMode
    utils.displayStudiesProgress(trainingModeHistories, path, 'Comparison Transfer Learning Mode')
    utils.saveStudyHistory(trainingModeStudy, trainingModeHistories, outFilePath)
    print("Backbone Transfer Learning Mode Selected - Pretrained:", pretrained, 'Trainable:', trainable)


    # ------------------------------------------
    # ---- Stage 2 - Compare Loss Functions ----
    # ------------------------------------------
    lossStudy = copy.deepcopy(trainingModeHistories[bestTrainMode]['config'])
    lossStudy['hyperparameters']["loss"] = losses
    lossesHistories = utils.researchStudies(trainDs, valDs, {bestTrainMode: lossStudy}, epochsSearch, verbose)
    lossesHistories[bestTrainMode+"_"+str(defaultHypers["loss"])] = trainingModeHistories[bestTrainMode]
    bestLoss = utils.getBestStudyFromHistories(lossesHistories)
    utils.displayStudiesProgress(lossesHistories, path, 'Comparison Loss Functions')
    utils.saveStudyHistory({'BestLoss': lossStudy}, lossesHistories, outFilePath)
    print("Best Loss Function:", str(lossesHistories[bestLoss]['config']['hyperparameters']["loss"]))
    
    # ------------------------------------------
    # ---- Stage 3 - Compare Learning Rates ----
    # ------------------------------------------
    learningRatesStudy = copy.deepcopy(lossesHistories[bestLoss]['config'])
    learningRatesStudy['hyperparameters']['learningRate'] = learningRates
    learningRatesHistories = utils.researchStudies(trainDs, valDs, {bestLoss: learningRatesStudy}, epochsSearch, verbose)
    learningRatesHistories[bestLoss+"_"+str(defaultHypers["learningRate"])] = lossesHistories[bestLoss]
    bestLearningRate = utils.getBestStudyFromHistories(learningRatesHistories)
    utils.displayStudiesProgress(learningRatesHistories, path, 'Comparison Learning Rate')
    utils.saveStudyHistory({'BestLearningRate': learningRatesStudy}, learningRatesHistories, outFilePath)
    print("Best Learning Rate:", str(learningRatesHistories[bestLearningRate]['config']["hyperparameters"]["learningRate"]))


    # --------------------------------------
    # ---- Stage 4 - Compare Optimizers ----
    # --------------------------------------
    optimizersStudy = copy.deepcopy(learningRatesHistories[bestLearningRate]['config'])
    optimizersStudy["hyperparameters"]["optimizer"] = optimizers
    optimizersHistories = utils.researchStudies(trainDs, valDs, {bestLearningRate : optimizersStudy}, epochsSearch, verbose)
    optimizersHistories[bestLearningRate+"_"+defaultHypers['optimizer'].__name__] = learningRatesHistories[bestLearningRate]
    bestOptimizier = utils.getBestStudyFromHistories(optimizersHistories)
    utils.displayStudiesProgress(optimizersHistories, path, 'Comparison Optimizer')
    utils.saveStudyHistory({'BestOptimizier': optimizersStudy}, optimizersHistories, outFilePath)
    print("Best Optimizier:", optimizersHistories[bestOptimizier]['config']["hyperparameters"]["optimizer"].__name__)
    
    
    # ------------------------------------------
    # ---- Stage 5 - Compare DNN Backbones -----
    # ------------------------------------------
    backboneConfigs = [backbone['backbone'](imgDim, weights=backbone['weights'], trainable=trainable) for backbone in backbonesForSearch]
    bestBackboneStudy = copy.deepcopy(optimizersHistories[bestOptimizier]['config'])
    bestBackboneStudy["architecture"]["backbone"] = backboneConfigs
    bestBackboneHistories = utils.researchStudies(trainDs, valDs, {bestOptimizier: bestBackboneStudy}, epochsSearch, verbose)
    bestBackboneHistories[bestOptimizier+"_"+defaultBackbone.__name__] = optimizersHistories[bestOptimizier]
    bestBackbone = utils.getBestStudyFromHistories(bestBackboneHistories)
    utils.displayStudiesProgress(bestBackboneHistories, path, 'Comparison DNN Backbone')
    utils.saveStudyHistory({'BestBackbone': bestBackboneStudy}, bestBackboneHistories, outFilePath)
    print("Best DNN Backbone:", bestBackboneHistories[bestBackbone]['config']["architecture"]["backbone"].name)

    # ---------------------------------------------------------
    # ---- Stage 6 - Train Best model on train+validation -----
    # ---------------------------------------------------------
    bestModelConfiguration = bestBackboneHistories[bestBackbone]['config']
    trainValDs = trainDs.concatenate(valDs)

    bestModel = FPML(**bestModelConfiguration["architecture"]).create(**bestModelConfiguration["hyperparameters"])
    bestModelHistory = bestModel.fit(trainValDs, validation_data=testDs, epochs=epochsBest, verbose=verbose)
    toSave = {'Best Model':{'history' : bestModelHistory, 'config' : bestModelConfiguration}}
    utils.displayStudiesProgress(toSave, path, 'Best Model')
    utils.saveStudyHistory({'Best Model': bestModelConfiguration}, toSave, outFilePath)
    utils.displayConfusion(testDs, bestModel, path)
    
    return bestModel


##### **Datasets Configuration**

In [ ]:
datasetsConfig = {
    'batchSize': 32, 
    'parallelTune': tf.data.AUTOTUNE, 
    'split': [0.7, 0.15, 0.15], 
    'inputDim': (224, 224, 3), 
    'seed': 9, 
    'shuffle': True
    }

#### **1. Gender Study** 
##### **Datasets**
* **SOCOFing**
* **NIST Special Database 4**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *binary-crossentropy, binary-focal-crossentropy, hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*


In [ ]:
genderDatasets = [
    ds.SOCOFingGender(**datasetsConfig, sampling=ds.SOCOFingGender.UNDER_SAMPLING),
    ds.NISTSDB4Gender(**datasetsConfig)
]

GenderConfig = {
    'defaultBackbone': backbones.MobileNetV2,
    'defaultHypers': {
        "optimizer": Adam,
        "learningRate": 0.001,
        "loss": 'binary_crossentropy',
        "metrics": 'accuracy'
    },
    'losses': ['binary_focal_crossentropy', 'hinge'],
    'learningRates': [0.1, 0.01, 0.0001, 0.00001],
    'optimizers': [Nadam, RMSprop],
    'backbonesForSearch': [
        {'backbone' : backbones.ResNet50, 'weights' : weightsRN50},
        {'backbone' : backbones.EfficientNetB2, 'weights' : weightsENB2},
        {'backbone' : backbones.InceptionV3, 'weights' : weightsINCEPTIONV3},
        {'backbone' : backbones.Xception, 'weights' : weightsXCEPTION}, 
    ],
    'epochsSearch': 2,
    'epochsBest': 2,
    'verbose': 0
}

genderOutPath = dbStudyOutPath.format('Gender/{}')

genderDatasetsPrepared = utils.prepareDatasetsForStudy(genderDatasets, genderOutPath) 


#### **2. Finger Name** 
##### **Datasets:**
* **SOCOFing**
* **NIST Special Database 4**
* **NIST Special Database 300a Roll**
* **NIST Special Database 300b - All Scanners**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *categorical-crossentropy, mean-squared-error, categorical-hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [ ]:
fingersNameDatasets = [
    ds.SOCOFingFingers(**datasetsConfig), 
    ds.NISTSDB4Fingers(**datasetsConfig), 
    ds.NISTSDB300aFingers(**datasetsConfig), 
    ds.NISTSDB302bFingers(**datasetsConfig)
    ]

fingerNameStudyParams = {
    'defaultBackbone': backbones.MobileNetV2,
    'defaultHypers': {
        "optimizer": Adam,
        "learningRate": 0.001,
        "loss": 'categorical_crossentropy',
        "metrics": 'accuracy'
    },
    'optimizers': [Nadam, RMSprop],
    'learningRates': [0.1, 0.01, 0.0001, 0.00001],
    'losses': ['mean_squared_error', 'categorical_hinge'],
    'backbonesForSearch': [
        {'backbone' : backbones.ResNet50, 'weights' : weightsRN50},
        {'backbone' : backbones.EfficientNetB2, 'weights' : weightsENB2},
        {'backbone' : backbones.InceptionV3, 'weights' : weightsINCEPTIONV3},
        {'backbone' : backbones.Xception, 'weights' : weightsXCEPTION}, 
    ],
    'epochsSearch': 5,
    'epochsBest': 10,
    'verbose': 1
}

fingerOutPath = dbStudyOutPath.format('FingerName/{}')
fingersDatasetsPrepared = utils.prepareDatasetsForStudy(fingersNameDatasets, fingerOutPath) 




In [ ]:
def researchStudy(preparedDatasets, studyParams):
    bestModels = {}
    for dsName, dsObj in preparedDatasets.items():
        bestModels[dsName] = researchBestModel(**dsObj, **studyParams)
    return bestModels
        
fingersBestModels = researchStudy(fingersDatasetsPrepared, fingerNameStudyParams)